<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/2020m0/9%20Preparation/Redlich_Kwong_CP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## setting up colab to pretty-print sympy formulae
from sympy          import init_printing
from sympy.printing import latex

def colab_LaTeX_printer(exp, **options):  
    from google.colab.output._publish import javascript 

    url_ = "https://colab.research.google.com/static/mathjax/MathJax.js?"
    cfg_ = "config=TeX-MML-AM_HTMLorMML" # "config=default"

    javascript(url=url_+cfg_)

    return latex(exp, **options)
# end of def
init_printing(use_latex="mathjax", latex_printer=colab_LaTeX_printer) 


## 宿題: van der Waals関数の臨界点
## Homework: The critical point of van der Waals equation

微分ができ、求解ができるので、変曲点を求めるのは簡単なはず。vdWの状態方程式は次のように書けます。

We learnt the way to differentiate and solve the equations.  Therefore it must be used to find the critical point of the van der Waals equation as the inflection point of the function.

In [ ]:
from sympy import *
from sympy.abc import *
(p+a/V**2)*(V-b)-k*T

               ⎛    a ⎞
-T⋅k + (V - b)⋅⎜p + ──⎟
               ⎜     2⎟
               ⎝    V ⎠

状態方程式の変曲点は、熱力学では臨界点と呼びます。体積変化に対して圧力が変動しないということは、圧縮率が無限大になり、体積が大きくゆらぐことを示唆しています。圧縮率が無限大ということはまた、音速が0になることを意味します。

The inflection point of the state equation is called the critical point in thermodynamics.
As we see above, $\partial p/\partial V$ becomes zero, i.e.,  the compressibility $\partial V/\partial p$ becomes infinite.
At the critical point, the volume fluctuates greatly, and the sound velocity becomes zero.

* van der Waals状態方程式の臨界点を求めて下さい。
* それができたら、Redlich-Kwong状態方程式の臨界点に挑戦して下さい。(解けないかもしれません)

In [ ]:
from sympy import *
from sympy.abc import *
solve([(k*T/(V-b)-(a/V**2)).diff(V),(k*T/(V-b)-(a/V**2)).diff(V,2)],[V,T])

⎡⎛      8⋅a  ⎞⎤
⎢⎜3⋅b, ──────⎟⎥
⎣⎝     27⋅b⋅k⎠⎦

In [ ]:
(k*T/(V-b)-(a/V**2)).subs([(V,3*b),(T,8*a/(27*b*k))])

  a  
─────
    2
27⋅b 

In [ ]:
from sympy import *
from sympy.abc import *
solve([(k*T/(V-b)-(a/(T**0.5*V*(V+b)))).diff(V),(k*T/(V-b)-(a/(T**0.5*V*(V+b)))).diff(V,2)],[V,T])

KeyboardInterrupt: ignored

In [ ]:
from sympy import *
from sympy.abc import *
p=k*T/(V-b)-a/(sqrt(T)*V*(V+b))
p1=p.diff(V)
p2=p.diff(V,2)

In [ ]:
# 分数を避ける。
pp1 = (p1*(V*(V-b)*(V+b))**2*sqrt(T)).simplify()
pp1

   3/2  2          2                2              2
- T   ⋅V ⋅k⋅(V + b)  + 2⋅V⋅a⋅(V - b)  + a⋅b⋅(V - b) 

In [ ]:
pp2 = (p2*(V*(V-b)*(V+b))**3*sqrt(T)).simplify()
pp2

   3/2  3          3      2          3                  3              3      
2⋅T   ⋅V ⋅k⋅(V + b)  - 4⋅V ⋅a⋅(V - b)  - 2⋅V⋅a⋅b⋅(V - b)  - 2⋅a⋅(V - b) ⋅(V + 

  2
b) 

In [ ]:
sol = solve([pp1,pp2],(a,b))
sol

⎡         ⎛ 3/2     ⎛    3 ___    2/3⎞                ⎞  ⎛ 3/2     ⎛   2/3   3
⎢         ⎜T   ⋅V⋅k⋅⎝1 + ╲╱ 2  + 2   ⎠    ⎛     3 ___⎞⎟  ⎜T   ⋅V⋅k⋅⎝- 2    - ╲
⎢(0, -V), ⎜───────────────────────────, V⋅⎝-1 + ╲╱ 2 ⎠⎟, ⎜────────────────────
⎣         ⎝             3                             ⎠  ⎝                    

 ___       3 ___         2/3     ⎞                             ⎞  ⎛ 3/2     ⎛ 
╱ 2  + 2 - ╲╱ 2 ⋅√3⋅ⅈ + 2   ⋅√3⋅ⅈ⎠    ⎛     3 ___ ⎛  1   √3⋅ⅈ⎞⎞⎟  ⎜T   ⋅V⋅k⋅⎝-
──────────────────────────────────, V⋅⎜-1 + ╲╱ 2 ⋅⎜- ─ - ────⎟⎟⎟, ⎜───────────
      6                               ⎝           ⎝  2    2  ⎠⎠⎠  ⎝           

  2/3   3 ___        2/3        3 ___     ⎞                             ⎞⎤
 2    - ╲╱ 2  + 2 - 2   ⋅√3⋅ⅈ + ╲╱ 2 ⋅√3⋅ⅈ⎠    ⎛     3 ___ ⎛  1   √3⋅ⅈ⎞⎞⎟⎥
───────────────────────────────────────────, V⋅⎜-1 + ╲╱ 2 ⋅⎜- ─ + ────⎟⎟⎟⎥
               6                               ⎝           ⎝  2    2  ⎠⎠⎠⎦

In [ ]:
sol[1]

⎛ 3/2     ⎛    3 ___    2/3⎞                ⎞
⎜T   ⋅V⋅k⋅⎝1 + ╲╱ 2  + 2   ⎠    ⎛     3 ___⎞⎟
⎜───────────────────────────, V⋅⎝-1 + ╲╱ 2 ⎠⎟
⎝             3                             ⎠

In [ ]:
Vc = solve(sol[1][1]-b, V)[0]
Vc

    b     
──────────
     3 ___
-1 + ╲╱ 2 

In [ ]:
Tc=solve(sol[1][0]-a,T)[0].subs(V,Vc)
Tc

                                           2/3
 2/3 ⎛                 a                  ⎞   
3   ⋅⎜────────────────────────────────────⎟   
     ⎜             3 ___          2/3     ⎟   
     ⎜   b⋅k       ╲╱ 2 ⋅b⋅k     2   ⋅b⋅k ⎟   
     ⎜────────── + ────────── + ──────────⎟   
     ⎜     3 ___        3 ___        3 ___⎟   
     ⎝-1 + ╲╱ 2    -1 + ╲╱ 2    -1 + ╲╱ 2 ⎠   

In [ ]:
pc=p.subs(V,Vc).subs(T,Tc).simplify()
pc

                         __________________
 2/3   ⎛       3 ___⎞ 3 ╱     3 ___    2/3 
3   ⋅a⋅⎝-5 + 4⋅╲╱ 2 ⎠⋅╲╱  1 + ╲╱ 2  + 2    
───────────────────────────────────────────
          _____    ____________            
    2    ╱  a   3 ╱      3 ___  ⎛    3 ___⎞
 3⋅b ⋅3 ╱  ─── ⋅╲╱  -1 + ╲╱ 2  ⋅⎝1 + ╲╱ 2 ⎠
      ╲╱   b⋅k                             